In [3]:
from dataclasses import dataclass
from typing import List, Tuple

# =========================
#   DATA MODEL
# =========================

@dataclass
class Job:
    name: str
    stages: List[str]
    C: List[float]
    deadline: float

    def total_C(self) -> float:
        return sum(self.C)


# =========================
#   HELPER FUNCTIONS
# =========================

def _check_same_length(job_i: Job, job_k: Job):
    if not (len(job_i.stages) == len(job_k.stages) ==
            len(job_i.C) == len(job_k.C)):
        raise ValueError(
            f"Jobs {job_i.name} and {job_k.name} must have "
        )

def compute_Mik(job_i: Job, job_k: Job) -> int:
    
    _check_same_length(job_i, job_k)

    Mi_k = 0
    in_segment = False

    for (s_i, c_i), (s_k, c_k) in zip(zip(job_i.stages, job_i.C),
                                      zip(job_k.stages, job_k.C)):
        shared = (s_i == s_k and c_i > 0.0 and c_k > 0.0)
        if shared and not in_segment:
            Mi_k += 1
            in_segment = True
        elif not shared:
            in_segment = False

    return Mi_k

def compute_Cimax(job_i: Job, job_k: Job) -> float:
    
    _check_same_length(job_i, job_k)

    cmax = 0.0
    for (s_i, c_i), (s_k, c_k) in zip(zip(job_i.stages, job_i.C),
                                      zip(job_k.stages, job_k.C)):
        if s_i == s_k and c_i > 0.0 and c_k > 0.0:
            if c_i > cmax:
                cmax = c_i
    return cmax


# =========================
#   RESPONSE-TIME BOUNDS
# =========================

def response_time_preemptive(job_k: Job, HP: List[Job]) -> float:
    
    N = len(job_k.stages)
    Ck = job_k.total_C()

    interference = 0.0
    carry_in = [0.0 for _ in range(max(N - 1, 0))]

    for job_i in HP:
        _check_same_length(job_i, job_k)

        Mi_k = compute_Mik(job_i, job_k)
        if Mi_k > 0:
            Ci_max = compute_Cimax(job_i, job_k)
            interference += 2.0 * Ci_max * Mi_k

        # carry-in term (stages 0..N-2)
        for j in range(N - 1):
            if job_k.C[j] <= 0.0:
                continue  # J_k does not use this stage
            if (job_i.stages[j] == job_k.stages[j] and
                job_i.C[j] > 0.0):
                if job_i.C[j] > carry_in[j]:
                    carry_in[j] = job_i.C[j]

    return Ck + interference + sum(carry_in)


def response_time_nonpreemptive(job_k: Job,
                                HP: List[Job],
                                ALL_jobs: List[Job]) -> float:
    
    N = len(job_k.stages)
    Ck = job_k.total_C()

    # Term 1: interference from HP
    interference_HP = 0.0
    carry_in = [0.0 for _ in range(max(N - 1, 0))]

    for job_i in HP:
        _check_same_length(job_i, job_k)

        Mi_k = compute_Mik(job_i, job_k)
        if Mi_k > 0:
            Ci_max = compute_Cimax(job_i, job_k)
            interference_HP += Ci_max * Mi_k

        # carry-in on stages 0..N-2
        for j in range(N - 1):
            if job_k.C[j] <= 0.0:
                continue
            if (job_i.stages[j] == job_k.stages[j] and
                job_i.C[j] > 0.0):
                if job_i.C[j] > carry_in[j]:
                    carry_in[j] = job_i.C[j]

    # Term 3: blocking (conservative) over ALL jobs at each stage
    blocking = [0.0 for _ in range(N)]
    for job_i in ALL_jobs:
        _check_same_length(job_i, job_k)

        for j in range(N):
            if job_k.C[j] <= 0.0:
                continue
            if (job_i.stages[j] == job_k.stages[j] and
                job_i.C[j] > 0.0):
                if job_i.C[j] > blocking[j]:
                    blocking[j] = job_i.C[j]

    return Ck + interference_HP + sum(carry_in) + sum(blocking)


# =========================
#   OPA (Optimal Priority Assignment)
# =========================

def opa_priority_assignment(
    jobs: List[Job],
    mode: str = "preemptive"
) -> Tuple[List[Job], List[Job]]:
    if mode not in ("preemptive", "nonpreemptive"):
        raise ValueError("mode must be 'preemptive' or 'nonpreemptive'")

    # For non-preemptive bound, ALL_jobs is fixed = original set
    ALL_jobs = list(jobs)

    U = list(jobs)    # unassigned (candidate higher priority)
    P: List[Job] = [] # assigned (lower priority in final result)

    while U:
        found = False

        for job_k in list(U):
            HP = [j for j in U if j is not job_k]  # in OPA, U\{k} treated as higher priority

            if mode == "preemptive":
                Rk = response_time_preemptive(job_k, HP)
            else:
                Rk = response_time_nonpreemptive(job_k, HP, ALL_jobs)

            if Rk <= job_k.deadline:
                # Job k can safely get the current lowest priority
                U.remove(job_k)
                P.insert(0, job_k)  # insert at front so P is high → low order
                found = True
                break

        if not found:
            # No safe lowest-priority job exists: stop with partial assignment
            return P, U

    return P, []



In [5]:

jobs = [
    Job(
        name="J1",
        stages=["R1", "S1"],
        C=[1.5, 3.0],
        deadline=14.0,
    ),
    Job(
        name="J2",
        stages=["R2", "S2"],
        C=[2.0, 2.5],
        deadline=16.0,
    ),
    Job(
        name="J3",
        stages=["R3", "S3"],
        C=[1.0, 4.0],
        deadline=12.0,
    ),
    Job(
        name="J4",
        stages=["R4", "S4"],
        C=[2.5, 3.0],
        deadline=20.0,
    ),
    Job(
        name="J5",
        stages=["R5", "S5"],
        C=[1.0, 3.5],
        deadline=14.0,
    ),
    Job(
        name="J6",
        stages=["R6", "S6"],
        C=[2.0, 3.0],
        deadline=16.0,
    ),
    Job(
        name="J7",
        stages=["R7", "S7"],
        C=[1.5, 2.5],
        deadline=13.0,
    ),
    Job(
        name="J8",
        stages=["R8", "S8"],
        C=[2.5, 4.0],
        deadline=20.0,
    ),
    Job(
        name="J9",
        stages=["R9", "S9"],
        C=[1.0, 2.0],
        deadline=15.0,
    ),
    Job(
        name="J10",
        stages=["R10", "S10"],
        C=[2.0, 3.5],
        deadline=180.0,
    ),
    Job(
        name="J11",
        stages=["R1", "S5"],
        C=[1.5, 2.5],
        deadline=180.0,
    ),
    Job(
        name="J12",
        stages=["R2", "S6"],
        C=[2.0, 3.0],
        deadline=180.0,
    ),
    Job(
        name="J13",
        stages=["R3", "S7"],
        C=[1.0, 3.5],
        deadline=180.0,
    ),
    Job(
        name="J14",
        stages=["R4", "S8"],
        C=[2.5, 2.5],
        deadline=180.0,
    ),
    Job(
        name="J15",
        stages=["R5", "S9"],
        C=[1.5, 4.0],
        deadline=180.0,
    ),
    Job(
        name="J16",
        stages=["R6", "S10"],
        C=[2.0, 2.0],
        deadline=180.0,
    ),
    Job(
        name="J17",
        stages=["R7", "S1"],
        C=[1.0, 3.0],
        deadline=180.0,
    ),
    Job(
        name="J18",
        stages=["R8", "S2"],
        C=[2.5, 3.5],
        deadline=180.0,
    ),
    Job(
        name="J19",
        stages=["R9", "S3"],
        C=[1.5, 2.0],
        deadline=180.0,
    ),
    Job(
        name="J20",
        stages=["R10", "S4"],
        C=[2.0, 4.0],
        deadline=180.0,
    ),
]



# ============================================================
# Helper to compute Rk for each assigned job
# ============================================================

def compute_R_values(P, mode, ALL_jobs):
    """
    Given final priority order P (HIGH → LOW), compute the worst-case delay R_k for each.
    """
    Rvals = {}

    # At the moment of assigning priority, HP = all jobs above in P
    for idx, job_k in enumerate(P):
        HP = P[:idx]    # all jobs higher priority than J_k

        if mode == "preemptive":
            Rk = response_time_preemptive(job_k, HP)
        else:
            Rk = response_time_nonpreemptive(job_k, HP, ALL_jobs)

        Rvals[job_k.name] = Rk

    return Rvals

# ============================================================
#  RUN OPA FOR PREEMPTIVE PIPELINE
# ============================================================

P_pre, U_pre = opa_priority_assignment(jobs, mode="preemptive")
print("\n================ PREEMPTIVE OPA ================")
print("Priority (HIGH → LOW): ", [j.name for j in P_pre])

if U_pre:
    print("UNSCHEDULABLE JOBS:", [j.name for j in U_pre])
else:
    # Compute worst-case delay for each assigned job
    Rvals = compute_R_values(P_pre, mode="preemptive", ALL_jobs=jobs)
    print("\nWorst-case delays (R_k):")
    for j in P_pre:
        print(f"  {j.name}: R = {Rvals[j.name]:.3f}   (deadline={j.deadline})")
    print("\nAll jobs schedulable (preemptive).")

# ============================================================
#  RUN OPA FOR NON-PREEMPTIVE PIPELINE
# ============================================================

P_non, U_non = opa_priority_assignment(jobs, mode="nonpreemptive")
print("\n============== NON-PREEMPTIVE OPA ==============")
print("Priority (HIGH → LOW): ", [j.name for j in P_non])

if U_non:
    print("UNSCHEDULABLE JOBS:", [j.name for j in U_non])
else:
    # Compute worst-case delay for each assigned job
    Rvals = compute_R_values(P_non, mode="nonpreemptive", ALL_jobs=jobs)
    print("\nWorst-case delays (R_k):")
    for j in P_non:
        print(f"  {j.name}: R = {Rvals[j.name]:.3f}   (deadline={j.deadline})")
    print("\nAll jobs schedulable (non-preemptive).")




================ PREEMPTIVE OPA ================
Priority (HIGH → LOW):  ['J20', 'J19', 'J18', 'J17', 'J16', 'J9', 'J15', 'J4', 'J14', 'J7', 'J13', 'J6', 'J2', 'J12', 'J1', 'J11', 'J10', 'J8', 'J5', 'J3']

Worst-case delays (R_k):
  J20: R = 6.000   (deadline=180.0)
  J19: R = 3.500   (deadline=180.0)
  J18: R = 6.000   (deadline=180.0)
  J17: R = 4.000   (deadline=180.0)
  J16: R = 4.000   (deadline=180.0)
  J9: R = 7.500   (deadline=15.0)
  J15: R = 9.500   (deadline=180.0)
  J4: R = 13.500   (deadline=20.0)
  J14: R = 12.500   (deadline=180.0)
  J7: R = 7.000   (deadline=13.0)
  J13: R = 9.500   (deadline=180.0)
  J6: R = 11.000   (deadline=16.0)
  J2: R = 11.500   (deadline=16.0)
  J12: R = 17.000   (deadline=180.0)
  J1: R = 10.500   (deadline=14.0)
  J11: R = 8.500   (deadline=180.0)
  J10: R = 15.500   (deadline=180.0)
  J8: R = 19.000   (deadline=20.0)
  J5: R = 14.000   (deadline=14.0)
  J3: R = 12.000   (deadline=12.0)

All jobs schedulable (preemptive).

============== NON-